<br>
<br>
<br>

# Face recognition with `Siamese Network:`

<br>
<br>

<br>

# 1. Import Dependencies: 

<br>


**Note:** Here, we will use `The Functional Api` In tensorflow.  <br>
`The Keras functional API is a way to create models that are more flexible than the keras.Sequential API.`
<br>

[documentation](https://www.tensorflow.org/guide/keras/functional_api)


In [2]:

# import standard dependencies:

import cv2 as cv 
import os
import random
import numpy as np 
import matplotlib.pyplot as plt 

# import tensorflow dependencies:


